In [15]:

import socket
import numpy as np
from PIL import Image
import pickle
import threading
import requests
import cv2
from google.colab.patches import cv2_imshow
import time


def send_all(sock, data):
    data = pickle.dumps(data)
    sock.sendall(len(data).to_bytes(4, byteorder='big'))
    sock.sendall(data)

def receive_all(sock):
    length = int.from_bytes(sock.recv(4), byteorder='big')
    data = b''
    while len(data) < length:
        packet = sock.recv(4096)
        if not packet:
            break
        data += packet
    return pickle.loads(data)

#kod z 1 zadania
def filtr(image_part):
    if len(image_part.shape) == 3:
        image_part = np.dot(image_part[...,:3], [0.2989, 0.5870, 0.1140]).astype(np.uint8)

    x = np.asarray([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    y = np.asarray([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])

    h, w = image_part.shape
    output = np.zeros((h, w), dtype=np.float64)

    for i in range(1, h-1):
        for j in range(1, w-1):
            gx = np.sum(x * image_part[i-1:i+2, j-1:j+2])
            gy = np.sum(y * image_part[i-1:i+2, j-1:j+2])
            output[i, j] = np.sqrt(gx**2 + gy**2)

    output = (output / output.max() * 255).astype(np.uint8)
    return output

def split_image(image, n_parts):
    return np.array_split(np.array(image), n_parts)

def merge_image(fragments):
    return np.vstack(fragments)

#server
def start_server():
    print("Serwer startuje")


    image_url = "https://visitwroclaw.s3.eu-central-1.amazonaws.com/wp-content/uploads/2025/06/rynek-wroclaw-1180x700-jpg.jpg"
    response = requests.get(image_url)
    image = cv2.imdecode(np.asarray(bytearray(response.content), dtype=np.uint8), cv2.IMREAD_COLOR)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_pil = Image.fromarray(image_rgb)

    n_clients = 4
    fragments = split_image(image_pil, n_clients)

    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(('localhost', 2040))
    server_socket.listen(n_clients)

    print("Server otwarty na localhost:2040, czeka na połączenie klientów")


    clients = []
    processed_fragments = [None] * n_clients
    threads = []


    for i in range(n_clients):
        client_socket, client_address = server_socket.accept()
        clients.append((client_socket, i))
        print(f" Klient {i+1}/4 połączony")
    def handle_client(client_socket, client_index):
        try:

            send_all(client_socket, fragments[client_index])
            print(f" Wysłano fragment {client_index+1}")
            processed_fragment = receive_all(client_socket)
            processed_fragments[client_index] = processed_fragment
            print(f" Odebrano przetworzony fragment {client_index+1}")

            client_socket.close()
        except Exception as e:
            print(f"Coś sie popsulo {client_index+1}: {e}")

    print("Równoległe przetwarzanie...")
    for client_socket, client_index in clients:
        thread = threading.Thread(target=handle_client, args=(client_socket, client_index))
        thread.start()
        threads.append(thread)


    for thread in threads:
        thread.join()

    print("Scalanie wyników...")
    result_array = merge_image(processed_fragments)
    result_image = Image.fromarray(result_array)


    result_image.save("obrazzad2.png")

    print("WSZYSTKIE FRAGMENTY PRZETWORZONE RÓWNOLEGLE!")


    server_socket.close()
    print("Serwer zakończył pracę")



#klienci
def run_client(client_id):
    try:
        print(f" Klient {client_id} startuje")

        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client_socket.connect(('localhost', 2040))

        print(f" Klient {client_id} połączony z serwerem")

        # odbiera
        fragment = receive_all(client_socket)
        print(f" Klient {client_id} odebrał fragment: {fragment.shape}")
        processed_fragment = filtr(fragment)
        #print(f" Klient {client_id} przetworzył fragment")

        #wysyła
        send_all(client_socket, processed_fragment)
        print(f"Klient {client_id} przetworzył swój fragment i wysłał wynik")

        client_socket.close()
        print(f"Klient {client_id} zakończył pracę\n")

    except Exception as e:
        print(f" Cos sie popsulo {client_id}: {e}\n")






# Uruchom serwer w osobnym wątku
server_thread = threading.Thread(target=start_server)
server_thread.daemon = True
server_thread.start()
print("Server start")
time.sleep(3)

# Uruchom 4 klientów też w osobnych atkach
threads = []
print("Start klientów")
for i in range(4):
    client_thread = threading.Thread(target=run_client, args=(i+1,))
    client_thread.daemon = True
    client_thread.start()
    threads.append(client_thread)
    #time.sleep(1)


for thread in threads:
    thread.join(timeout=10)
time.sleep(2)
print("program sie nie zepsuł (chyba)")

Serwer startujeServer start

Server otwarty na localhost:2040, czeka na połączenie klientów
Start klientów
 Klient 1 startuje
 Klient 2 startuje
 Klient 3 startuje
 Klient 3 połączony z serwerem
 Klient 1/4 połączony
 Klient 2/4 połączony
 Klient 1 połączony z serwerem
 Klient 4 startuje
 Klient 2 połączony z serwerem
 Klient 3/4 połączony
 Klient 4 połączony z serwerem
 Klient 4/4 połączony
Równoległe przetwarzanie...
 Wysłano fragment 1
 Wysłano fragment 3
 Wysłano fragment 2
 Wysłano fragment 4
 Klient 2 odebrał fragment: (194, 1180, 3)
 Klient 3 odebrał fragment: (194, 1180, 3)
 Klient 1 odebrał fragment: (194, 1180, 3)
 Klient 4 odebrał fragment: (194, 1180, 3)
Klient 1 przetworzył swój fragment i wysłał wynik
Klient 1 zakończył pracę

 Odebrano przetworzony fragment 1
Klient 4 przetworzył swój fragment i wysłał wynik
Klient 4 zakończył pracę

 Odebrano przetworzony fragment 4
Klient 2 przetworzył swój fragment i wysłał wynik
Klient 2 zakończył pracę

 Odebrano przetworzony fragme